In [ ]:
# 사전 설치 라이브러리 : pip install selenium webdriver-manager bs4 openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
# 드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
# 검색어 설정
keyword = "파이썬"
search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}"

# 검색 페이지 접속
driver.get(search_url)
time.sleep(2)

# 뉴스 링크 수집
news_links = []
links = driver.find_elements(By.CSS_SELECTOR, "a.info")

# 네이버 뉴스 링크만 수집 (3개)
for link in links:
    if len(news_links) >= 3:  # 원하는 기사 수로 수정 가능
        break
    href = link.get_attribute('href')
    if href and 'news.naver.com' in href:
        news_links.append(href)

print(f"수집된 뉴스 링크 수: {len(news_links)}")

In [ ]:
# 뉴스 데이터를 저장할 리스트
news_data = []

# 각 뉴스 기사 내용 수집
for url in news_links:
    driver.get(url)
    time.sleep(2)

    # HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        # 제목 추출
        title = soup.select_one('#title_area span').text.strip()

        # 본문 추출
        content = soup.select_one('#dic_area').text.strip()

        # 데이터 저장
        news_data.append({
            'title': title,
            'content': content,
            'url': url
        })

        print(f"제목: {title}")
        print("-" * 50)

    except Exception as e:
        print(f"Error with {url}: {e}")

# 드라이버 종료
driver.quit()

In [ ]:
# DataFrame 생성
df = pd.DataFrame(news_data)

# 결과 확인
print("\n=== 수집된 뉴스 목록 ===")
for idx, row in df.iterrows():
    print(f"\n[기사 {idx+1}]")
    print(f"제목: {row['title']}")
    print(f"URL: {row['url']}")
    print("-" * 50)

# Excel 파일로 저장
df.to_excel(f'{keyword}_news.xlsx', index=False)
print(f"\n결과가 {keyword}_news.xlsx 파일로 저장되었습니다.")